# 第8回：モデルの改善
* 前回は機械学習（決定木とランダムフォレスト）で予測モデルを作成し、精度評価を行いランダムフォレストのほうが精度が良いことを確認しました。
* 実際に機械学習を活用する場合には1回でいきなり良い精度になることは少なく、試行錯誤をしながら精度を上げていくことがほとんどです。
* 今回はランダムフォレストを用いて、精度向上の取り組みとして以下の2テーマを行います。
    * ハイパーパラメータチューニング
    * 新たな特徴量の作成（予測に影響しそうな列を新たに作成）

# 1. データ読み込み
* まずはデータを読み込みましょう。
* 予測モデル作成用に整形したデータ（data_ml.csv）を読み込みます。

In [ ]:
import pandas as pd             # 表形式のデータ操作ライブラリ
import matplotlib.pyplot as plt # グラフ描画ライブラリ
import seaborn as sns           # グラフ描画ライブラリ

from sklearn.tree import DecisionTreeClassifier      # 決定木による分類のためのライブラリ
from sklearn.ensemble import RandomForestClassifier  # ランダムフォレストによる分類のためのライブラリ

from sklearn.metrics import roc_auc_score     # AUC値を算出するライブラリ
from sklearn.metrics import roc_curve         # ROC曲線を描画するためのライブラリ
from sklearn.metrics import accuracy_score    # 正解率を算出するライブラリ
from sklearn.metrics import confusion_matrix  # 混同行列を作成するライブラリ

from sklearn.model_selection import train_test_split # データを学習用と検証用に分割するライブラリ
from sklearn.model_selection import GridSearchCV     # グリッドサーチによるパラメータチューニングを実施するライブラリ

random_state = 1 # 乱数固定
sns.set() 

In [ ]:
# 警告（warning）を表示させない
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('data_ml.csv')

In [ ]:
print('データのサイズ：', data.shape) # データサイズ確認
display(data.head()) # データが正しく読み込めたことを確認

In [ ]:
# データを説明変数と目的変数に分割し、さらに学習用データと検証用データに分割
X = data.drop(['y'],axis=1)
y = data.y

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=random_state) 

# 2. ランダムフォレストの予測精度の再確認
* 前回の演習でも実施しましたが、ランダムフォレストの予測精度を再確認してみましょう。

## 2-1. 評価指標の表示を関数にまとめる
* 同じ処理を何度も実施する場合は、関数としてまとめておくと便利です。
* 関数を呼び出すことで、簡単に処理を実行できます。

In [ ]:
def pred_and_eval(model, X_test, y_test) :
    ## この関数では、学習済みモデル、検証用説明変数、検証用目的変数をもとに ##
    ## 正解率、AUC、ROC曲線、特徴量重要度を表示する関数です                 ##
    
    # 正解率を確認
    y_pred = model.predict(X_test)
    ac = accuracy_score(y_test, y_pred)
    print('正解率：{:.3f}'.format(ac))    
    
    # AUCを確認
    y_prob = model.predict_proba(X_test) # 0,1ではなく予測確率を出力
    auc = roc_auc_score(y_test,y_prob[:,1])
    print('AUC：{:.3f}'.format(auc))    
    
    # ROC曲線描画
    plt.rcParams["font.size"] = 18
    fpr, tpr, thresholds = roc_curve(y_test, y_prob[:,1])
    plt.figure(figsize=(5,5))
    plt.plot(fpr, tpr)
    plt.plot([0,1],[0,1],color="gray",linestyle="dotted")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    
    # 特徴量重要度を確認
    feature_importances = pd.DataFrame(model.feature_importances_,
                                  index = X_test.columns,
                                  columns=["importance"]).sort_values("importance",ascending=False)
    feature_importances.head(20).plot.bar(figsize=(12,4), fontsize=20)

## 2-2. ランダムフォレストの精度確認

In [ ]:
forest = RandomForestClassifier(random_state = random_state)  # この時点では未学習（常に同じ結果が出るようにrandom_stateで乱数固定）
forest.fit(X_train, y_train)   

pred_and_eval(forest, X_test, y_test) 

# 3. 精度改善その1：パラメータチューニング

* まずはランダムフォレストのパラメータを調整して精度向上できるか確認しましょう。
* 今回は以下の3つのパラメータについて順番に探索します。
    * 「木の深さ」max_depth
    * 「決定木の数」n_estimators
    * 「分岐に必要なデータ数」min_samples_leaf

## 3-1. max_depthの探索

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from tqdm import tqdm

In [ ]:
depth_list = [5,10,15,20,25,30]
aucs =[]
for depth in tqdm(depth_list): # 5, 10, ... とmax_depthを変えたときの精度を繰り返し確認
    rf = RandomForestClassifier(max_depth= depth , n_estimators=100, random_state=random_state) # この時点では未学習
    rf.fit(X_train, y_train)                  # 学習
    y_prob = rf.predict_proba(X_test)         # 0,1ではなく予測確率を出力
    auc = roc_auc_score(y_test,y_prob[:,1])   # AUCスコアを算出
    aucs.append(auc)                          # 算出したAUCを記録
    
plt.plot(depth_list,aucs) # max_depthごとのAUCを折れ線グラフで描画

## 3-2. n_estimatorsの探索

In [ ]:
param_list = [5,10,20,30,50,100,200,300]
aucs =[]
for param in tqdm(param_list):
    rf = RandomForestClassifier(max_depth= 15 , n_estimators=param, random_state=random_state)
    rf.fit(X_train, y_train) 
    y_prob = rf.predict_proba(X_test) # 0,1ではなく予測確率を出力
    auc = roc_auc_score(y_test,y_prob[:,1])
    aucs.append(auc)
plt.plot(param_list,aucs)

## min_samples_leafの探索

In [ ]:
param_list = [1,2,3,4,5,7,10,20]
aucs =[]
for param in tqdm(param_list):
    rf = RandomForestClassifier(max_depth= 15 , n_estimators=200, min_samples_leaf=param, random_state=random_state)
    rf.fit(X_train, y_train) 
    y_prob = rf.predict_proba(X_test) # 0,1ではなく予測確率を出力
    auc = roc_auc_score(y_test,y_prob[:,1])
    aucs.append(auc)
plt.plot(param_list,aucs)

## 3-4. 探索したパラメータで最終確認

In [ ]:
# rf = RandomForestClassifier(max_depth= 15, n_estimators=200, min_samples_leaf=3, random_state=random_state)
rf = RandomForestClassifier(max_depth= 15, n_estimators=200, min_samples_leaf=3, random_state=random_state)
rf.fit(X_train, y_train)

pred_and_eval(rf, X_test, y_test) 

# 4. 精度改善その2：特徴量作成
* 機械学習ではデータから自動でパターンを学習して予測モデルを作成できますが、学習させるデータに人間の知見を加えることで、より良いモデルが作成できます。
* 対象業界に詳しい人の知見を取り入れたり、第2回演習で実施したようなデータのグラフ化による仮設立案などから、新たな特徴量（列）を追加します。
* 実際に、予測精度の向上を目指して予測に効果がありそうな特徴量（列）を追加してみましょう。
* 今回は学習用データと検証用データを分割する前のデータでグラフ化しながら確認していますが、実際の業務で分析を行う際には、学習用データのみを使い仮設検討し、検証用データで精度検証を行います。

## 4-1. 曜日の追加
* データの年月日がわかっているので、曜日を表す列を追加することができます。
* 予測対象にもよりますが、「金曜日は居酒屋が混雑しやすい」など、曜日が売上や契約に関係するケースは多くあります。
* 実際に曜日を表す列を追加してみましょう。

#### year, month, dayを組み合わせて年月日列（date列）を追加

In [ ]:
data["date"]= pd.to_datetime(data["year"].astype(int).astype(str) + "/" + data["month"].astype(str) + "/" + data["day"].astype(str))

In [ ]:
print('データのサイズ：', data.shape) # データサイズ確認
display(data.head())                  # date列を作成できたことを確認

#### date列をもとに曜日列を追加

In [ ]:
data['day_name'] = data['date'].dt.day_name()  # day_name で曜日を追加

In [ ]:
print('データのサイズ：', data.shape) # データサイズ確認
display(data.head())                  # day_name列を追加できたことを確認

#### 曜日ごとに定期預金をした人としていない人の割合をグラフ化して確認

In [ ]:
col_name = 'day_name'
sns.set()               # グラフのデザインをSeabornライブラリのデフォルトスタイルに変更
sns.set_context('talk') # グラフの字の大きさを調整

# カテゴリ別にグラフ化するためにデータを加工
count_data = (data.groupby(col_name)['y']   # 集計対象の列でグループ化
              .value_counts(normalize=True) # 割合算出
              .rename('percentage') 
              .reset_index())

sns.catplot(x='y', y='percentage', col=col_name, kind='bar', data=count_data) # catplotで棒グラフを描画

* グラフでは曜日による違いは少なそうに見えます。

#### 曜日列を数値に変換

In [ ]:
# get_dummies機能を使い、day_name 列のOne-Hotエンコーディングを行います。
data = pd.get_dummies(data, columns = ['day_name'])

In [ ]:
# 曜日を追加できたので、一時的に追加した年月日列を削除します
data.drop("date",axis=1,inplace =True)

## 4-2. データを学習用データと検証用データに分割し、精度検証

In [ ]:
X = data.drop(['y'],axis=1)
y = data.y

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=random_state) 

In [ ]:
rf = RandomForestClassifier(max_depth= 15, n_estimators=200, min_samples_leaf=3, random_state=random_state)
rf.fit(X_train, y_train)

pred_and_eval(rf, X_test, y_test) 

# 5. 精度改善その3：新たなデータの追加（経済指標）
* 手元にあるデータだけでは、特徴量作成などの工夫による精度向上にも限界があります。
* そのような場合には、新しいデータを追加することで精度向上できる場合もあります。
* 今回のテーマは定期預金の契約予測なので、新たに経済指標データを追加してみましょう。

In [ ]:
economic = pd.read_csv("economic.csv")
economic['month'] = economic['month'].map( {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 
                                'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12} ).astype(int)
economic.head()

### 追加指数の説明
* year          ：該当年
* month         ：該当月
* emp.var.rate  ：雇用者数の変動率
* cons.price.idx：消費者物価指数
* cons.conf.idx ：消費者信頼感指数
* nr.employed   ：総雇用者数 
* euribor3m     ：ユーロ圏での銀行間取引金利（３か月）

In [ ]:
# dataとeconomicをyear列とmonth列が同じ所で結合する
data_ext = pd.merge(data,economic, on=["year","month"])
data_ext.head()

In [ ]:
X = data_ext.drop(['y'],axis=1)
y = data_ext.y

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=random_state) 

In [ ]:
rf = RandomForestClassifier(max_depth= 15, n_estimators=200, min_samples_leaf=3, random_state=random_state)
rf.fit(X_train, y_train)

pred_and_eval(rf, X_test, y_test) 